# KBMod Analysis Demo

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from kbmod.analysis.plot_results import *
from kbmod.result_list import *

%matplotlib inline

To run an analysis of KBMOD's output you need results in the form of either a `ResultList` or saved files. For the purposes of this notebook you can use that fake result data (corresponding to the `data/demo` images) included in `data/fake_results/results.ecsv`. These results were generated by the KBMOD_Demo notebook.

## Loading Results

We load all of the result data as a `ResultList` using the class's `read_table` function. We extract the first row to use for later examples.

In [ ]:
results = ResultList.read_table("../data/fake_results/results.ecsv")
print(f"Loaded {results.num_results()} results.")

row0 = results.results[0]

Note that in previous versions (<=1.1) KBMOD saved output was also saved in a series of individual files such as `all_ps_DEMO.npy` and `phi_DEMO.txt`. You can still load these using the `load_result_list_from_files` function:

```
results_dir = "../data/fake_results/"
results_suffix = "DEMO"
results = load_result_list_from_files(results_dir, results_suffix)
```

## Plotting the Results

We can use the `ResultsVisualizer` class to plot the different aspects of the `ResultList`.

We start by plotting the coadded stamp for the top result.

In [ ]:
ResultsVisualizer.plot_single_stamp(row0.stamp)

If we have multiple stamps from either all_stamps or from multiple trajectories, we can plot them in a grid. Here we use the per-time step stamps.

In [ ]:
ResultsVisualizer.plot_stamps(row0.all_stamps, columns=5)

We can visualize the time series such as: `psi_curve`, `phi_curve`, or `light_curve` using the `plot_time_series` function. If we provide timestamps (such as with the top and bottom figure), they are used to both label the x-axis and set spacing.

In [ ]:
fig1 = plt.figure(figsize=(10, 15))
(ax1, ax2, ax3) = fig1.subplots(3, 1)

ResultsVisualizer.plot_time_series(
    row0.light_curve, times=results.all_times, axes=ax1, title="Lightcurve with time spacing"
)

ResultsVisualizer.plot_time_series(row0.light_curve, axes=ax2, title="Lightcurve with equal spacing")

ResultsVisualizer.plot_time_series(
    row0.light_curve,
    times=results.all_times,
    indices=[0, 1, 2, 4, 5, 7, 8],
    axes=ax3,
    title="Lightcurve with time spacing and invalid indices",
)

Or we can put it altogether with the `plot_result_row` function.

In [ ]:
ResultsVisualizer.plot_result_row(row0)

# Histograms of Results

We can plot the histograms of individual attributes of the results. First we need to load a result set with more than one entry. Then we plot the histogram of number of observations in the trajectory.

In [ ]:
results2 = load_result_list_from_files("../data/fake_results_noisy/", "DEMO")
print(f"Loaded {results2.num_results()} results.")

counts = results2.get_result_values("obs_count")
_ = plt.hist(counts)

The `ResultsVisualizer` class provides a helper function to plotting a 2-d histogram of the trajectory's starting positions on the image. This requires knowing the image size. The demo data consists of 256 by 256 images.

In [ ]:
ResultsVisualizer.plot_starting_pixel_histogram(results2, 256, 256)